In [14]:
import requests
import json
import string
import pickle
import time
base_url = "https://api.github.com"
# https://github.com/settings/tokens
api_key = ""
save_folder = f"./"
with open('../api.json', 'r') as file:
    api_key = json.load(file)["GH_API_KEY"]
    
def pickle_this(target, name, file_path=save_folder):
    path = file_path+name
    file = open(path, "wb")
    if file:
        pickle.dump(target, file)
        file.close()
        return True
    file.close()
    return False
    

def unpickle_this(name, file_path=save_folder):
    path = file_path+name
    file = open(path, "rb")
    res = []
    if file:
        res = pickle.load(file)
        file.close()
        return res
    file.close()
    return res

base_url = "https://api.github.com"
# https://github.com/settings/tokens
def get_call(paths, params=None, header=None):
    url = f"{base_url}/{paths}"
    response = requests.get(url, params=params, headers=header)
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 403:
        print("La patience est d'or. Enculé")
        time.sleep(61)
        response = requests.get(url, params=params, headers=header)
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.json().get('message', 'Unknown error')}")
        return {}
    
def get_repos_created_in_day_multipages(year, month="01", day="01", max_pages=0):
    # Recherche de repositories créés en 2024
    query = "created:"+str(year)+"-"+str(month)+"-"+str(day)
    params = {"q": query, "per-page": 30}
    header = {"Authorization": f"token {api_key}"}
    # Appeler l'API de recherche de repositories
    repos_data = get_call("search/repositories", params=params, header=header)
    num_pages = 0
    res = []
    if repos_data:
        res.extend(repos_data.get("items", []))
        repo_pages = (repos_data.get("total_count", 0) // 30)
        print(repo_pages)
        if max_pages > 0:
            num_pages = min(max_pages, repo_pages)
        
        for ii in range(2, num_pages+1):
            repos_data={}
            params = {"q": query, "per-page": 30, "page": ii}
            repos_data = get_call("search/repositories", params=params, header=header)
            res.extend(repos_data.get("items", []))
            pickle_this(res, "repos_"+str(year)+"_"+str(month))
        
        return res
    else:
        return []

In [ ]:
# ajout de la boucle pour que cela fonctionne chaque jour avec de la robustesse 
URI_MONGO_DB = "https://api-base-fastapi-mpt-dkhjb0cydga2aydc.canadacentral-01.azurewebsites.net"
max_pages=25
months = ["01","02","03","04","05","06","07","08","09","10",11,12]
res = {}

for year in range(2024,2010,-1):
    for month in months:
        for day in range (1,28):
            print("date du jour : " + (str(day) if day>=10 else "0"+str(day)) +"-"+ str(month) +"-"+ str(year))
            res = get_repos_created_in_day_multipages(year, month=month, day=day if day>=10 else "0"+str(day), max_pages=max_pages)
            sortie = requests.post(f"{URI_MONGO_DB}/add_data", json=res)
            print("code : " + str(sortie.status_code))
print(len(res))

date du jour : 01-01-2024
2290
code : 200
date du jour : 02-01-2024
3471
La patience est d'or. Enculé
code : 200
date du jour : 03-01-2024
3723
La patience est d'or. Enculé
code : 200
date du jour : 04-01-2024
3756
La patience est d'or. Enculé
code : 200
date du jour : 05-01-2024
3559
La patience est d'or. Enculé
code : 200
date du jour : 06-01-2024
3057
code : 200
date du jour : 07-01-2024
La patience est d'or. Enculé
3059
code : 200
date du jour : 08-01-2024
4033
La patience est d'or. Enculé
code : 200
date du jour : 09-01-2024
4182
La patience est d'or. Enculé
code : 200
date du jour : 10-01-2024
4185
La patience est d'or. Enculé
code : 200
date du jour : 11-01-2024
4087
La patience est d'or. Enculé
code : 200
date du jour : 12-01-2024
3801
code : 200
date du jour : 13-01-2024
3066
code : 200
date du jour : 14-01-2024
3062
La patience est d'or. Enculé
code : 200
date du jour : 15-01-2024
4015
La patience est d'or. Enculé
code : 200
date du jour : 16-01-2024
4230
La patience est d'or

KeyboardInterrupt: 

In [23]:
import requests
import json
from collections import Counter
import pandas as pd

def get_call(paths,url, params=None, header=None):
    url = f"{url}/{paths}"
    response = requests.get(url, params=params, headers=header)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.json().get('message', 'Unknown error')}")
        return None
    
repositories=get_call(paths="start",url="https://api-github-recup-data-auto-mpt-g4fpatffcxd7dkhd.canadacentral-01.azurewebsites.net")
print(repositories)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
# ajout de la boucle pour que cela fonctionne chaque jour avec de la robustesse 
URI_MONGO_DB = "https://api-base-fastapi-mpt-dkhjb0cydga2aydc.canadacentral-01.azurewebsites.net"
max_pages=25
months = ["06","07","08","09","10",11,12]
res = {}
year = 2024
for month in months:
    for day in range (1,28):
        print("date du jour : " + (str(day) if day>=10 else "0"+str(day)) +"-"+ str(month) +"-"+ str(year))
        res = get_repos_created_in_day_multipages(year, month=month, day=day if day>=10 else "0"+str(day), max_pages=max_pages)
        sortie = requests.post(f"{URI_MONGO_DB}/add_data", json=res)
        print("code : " + str(sortie.status_code))
print(len(res))

date du jour : 01-06-2024
3359
code : 500
date du jour : 02-06-2024
3448
code : 200
date du jour : 03-06-2024
4566
code : 500
date du jour : 04-06-2024
4560
code : 500
date du jour : 05-06-2024
4859
code : 200
date du jour : 06-06-2024
4612
code : 200
date du jour : 07-06-2024
4274
code : 200
date du jour : 08-06-2024
3435
code : 200
date du jour : 09-06-2024
3395


KeyboardInterrupt: 

In [3]:
print(repositories[0].keys())

NameError: name 'repositories' is not defined

In [6]:
pickle_this(repositories, "repos")

NameError: name 'repositories' is not defined

In [7]:
results = unpickle_this("repos")

In [1]:
for ii in range(2010, 2026):
    if ii in dict.keys():
        repositories = get_repos_created_in_year(year=ii)
        # Analyser les langages utilisés
        if repositories:
            pickle_this(repositories, "repos"+ii)
            results[str(ii)] = get_languages_from_repositories(repositories)
        else:
            print(f"Aucun repository trouvé pour %n.", ii)
print(results)

TypeError: unbound method dict.keys() needs an argument

In [14]:
len(results)

30

In [8]:
def get_last_page_link(link_str):
        link_last = ""
        num_last = 1
    for page in response["link"].split(", "):
        page_split = page.split("; ")
        link_last = page_split[0][1:-1]
        if "last" in page_split[1]:
            num_last = link_last.split("&page=")
    link_last = link_last+"&page="
    return [link_last, num_last]

IndentationError: unindent does not match any outer indentation level (<string>, line 4)

In [2]:
def fetch_github_data():
    max_pages = 25
    months = [f"{i:02d}" for i in range(1, 13)]
    res = []

    for year in range(2010, 2025):£
        for month in months:
            for day in range(1, 29):
                day_str = f"{day:02d}"
                repos = get_repos_created_in_day_multipages(year, month, da_str, max_pages)
                if URI_MONGO_DB:
                    requests.post(f"{URI_MONGO_DB}/add_data", json=repos)
                res.extend(repos)

    return {"total_repos": len(res)}

SyntaxError: invalid character '£' (U+00A3) (3205640200.py, line 6)